In [18]:
import os
import copy
import nltk
import json
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from stop_words import get_stop_words
from nltk.tokenize import TweetTokenizer
import re

In [19]:
all_stopwords = []
all_stopwords.extend(get_stop_words("ukrainian"))
all_stopwords.extend(get_stop_words("english"))
all_stopwords.extend(get_stop_words("russian"))

In [20]:
categories = dict()
with open("list_of_JSONs.txt", 'r', encoding='utf-8') as f:
    for i in f.readlines():
        if " = " in i:
            tup = i.strip().split(" = ")
            categories[tup[0]] = tup[1]
print(categories)            

{'Добкин Михаил': '@mdobkin_ver3.0.json', 'Олександр Доній': '@oles_doniy_ver3.0.json', 'Уляна Супрун': 'dataSuprunEXT.json', 'УНА-УНСО': '@unaunso_ver3.0.json', 'Віталій Кличко': '@Vitaliy_Klychko_ver3.0.json', 'Олександра Кужель': '@KuzhelUA_ver3.0.json', 'Арсен Аваков': '@AvakovArsen_ver3.0.json', 'Наталія Королевська': '@NKorolevska_ver3.0.json', 'Сергій Одарич': '@sergiy_odarych_ver3.0.json', 'Анатолій Гриценко': '@agrytsenko_ver3.0.json', 'Іван Плачков': '@ivan_plachkov_ver3.0.json', 'Сергей Власенко': '@Sergey_Vlasenko_ver3.0.json', 'Юлія Тимошенко': '@YuliaTymoshenko_ver3.0.json', 'Арсеній Яценюк': '@Yatsenyuk_AP_ver3.0.json', 'Олександр Аронець': '@aronets_ver3.0.json', 'Андрій Парубій': '@AndriyParubiy_ver3.0.json', 'Віктор Балога': '@ViktorBaloha_ver3.0.json', 'Майкл Щур': 'dataMichaelSchur.json', 'Андрій Шевченко': '@AShevch_ver3.0.json', 'Андрій Садовий': '@AndriySadovyi_ver3.0.json', 'Іван Куліченко': '@IvanKulichenko_ver3.0.json', 'Роман Скрипін': 'dataSkrypinEXT.json', 

In [21]:
def preprocess(file):
    cur_tweets = file.split('"]["')
    for i in range(len(cur_tweets)):
#         print(cur_tweets[i])
        # З Прем'єр-міністром Баварії Хорстом Зеєхофером говорили про збереження санкційного тиску на Москву https://t.co/r0RlPemAUH
        cur_tweets[i] = cur_tweets[i].replace("\\n", "").replace("'","").lower()
        links = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', cur_tweets[i])
        for link in links:
            cur_tweets[i] = cur_tweets[i].replace(link, "")
        cur_tweets[i] = ''.join([k for k in cur_tweets[i] if not k.isdigit()])
        tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) #nltk.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(cur_tweets[i])
        cur_tweets[i] = " ".join([w for w in tokens if w not in all_stopwords])
#         print("--", cur_tweets[i])
        # премєр-міністром баварії хорстом зеєхофером говорили збереження санкційного тиску москву

    return cur_tweets

In [22]:
def read_all_files(src):  # src - dict()
    docs, labels = [], []
    for k, v in src.items():
        with open("JSONs/" + v, 'r', encoding='utf-8') as tweets:
            docs.append(preprocess(tweets.read()[2:]))
            labels.append(k)
    return docs, labels
#         break
all_tweets, all_labels = read_all_files(categories)   

In [23]:
# print(all_tweets)
print(len(all_tweets))

41


In [24]:
print(all_labels)

['Добкин Михаил', 'Олександр Доній', 'Уляна Супрун', 'УНА-УНСО', 'Віталій Кличко', 'Олександра Кужель', 'Арсен Аваков', 'Наталія Королевська', 'Сергій Одарич', 'Анатолій Гриценко', 'Іван Плачков', 'Сергей Власенко', 'Юлія Тимошенко', 'Арсеній Яценюк', 'Олександр Аронець', 'Андрій Парубій', 'Віктор Балога', 'Майкл Щур', 'Андрій Шевченко', 'Андрій Садовий', 'Іван Куліченко', 'Роман Скрипін', 'Євгеній Суслов', 'Михаил Бродский', 'Олег Ляшко', 'Олександр Турчинов', 'ВО Свобода', 'Павло Шеремета', 'Петро Порошенко', "В'ячеслав Кириленко", 'Святослав Вакарчук', 'Виктор Пилипишин', 'Олег Тягнибок', 'ВО Батьківщина', 'Петро Мельник', 'Зорян Шкіряк', 'Костянтин Грищенко', 'Михайло Саакашвілі', 'Леся Оробець', 'Володимир Арієв', 'Михайло Ткач']


In [25]:
gen = [" ".join(i) for i in all_tweets]
# print(gen)

In [26]:
vect = TfidfVectorizer(max_features=650, analyzer="word").fit(gen)

In [27]:
ind = 0
gl_tr_d, gl_te_d, gl_tr_l, gl_te_l = [], [], [], []
for tweets in all_tweets:
    test_size = int(0.4 * len(tweets))
    train_size = len(tweets) - test_size
    tr_d = [tweets[i] for i in range(train_size)]
    te_d = [tweets[i] for i in range(test_size)]

    gl_tr_d.append(" ".join(tr_d))
    gl_te_d.append(" ".join(te_d))
    gl_tr_l.append(all_labels[ind])
    gl_te_l.append(all_labels[ind])
    ind += 1
#     break
# print(gl_tr_l)
# print(gl_tr_d)

In [28]:
train_data = [vect.transform([" ".join(text.split())]).toarray()[0] for text in gl_tr_d]

In [29]:
test_data = [vect.transform([" ".join(text.split())]).toarray()[0] for text in gl_te_d]

### Multinomial Naive Bayes

In [30]:
clf_MNB = MultinomialNB()

In [31]:
clf_MNB.fit(train_data, gl_tr_l)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
clf_MNB.score(train_data, gl_tr_l)

1.0

In [34]:
# print(gl_te_l)
pred_MNB_test = clf_MNB.predict(test_data)
score_MNB_test = accuracy_score(gl_te_l, pred_MNB_test)
print(score_MNB_test)

1.0


### SVM 

In [35]:
clf_svm = svm.SVC(C=3.0, kernel="linear")

In [36]:
clf_svm.fit(train_data, gl_tr_l)

SVC(C=3.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
clf_svm.score(train_data, gl_tr_l)

1.0

In [72]:
new_tweet = 'Видео \"Дохід УЄФА від проведення Евро 2012\" (http://t.co/dm9o03AT) на @YouTube загружено.'

In [73]:
new_tweet = preprocess(new_tweet)
print(new_tweet)

['видео " дохід уєфа проведення евро " ( загружено .']


In [78]:
vectorized = [vect.transform([" ".join(new_tweet)]).toarray()[0]]
# vectorized

In [77]:
# vectorized
clf_MNB.predict(vectorized)

# clf_svm.score(vectorized, ["Петро Мельник"])

array(['Петро Мельник'],
      dtype='<U19')